In [1]:
#سلول 1:
!pip install -q transformers seqeval scikit-learn pandas numpy torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.4 MB/s eta 0:00:00


In [3]:
#سلول2:
from google.colab import drive
import zipfile

drive.mount('/content/drive')
zip_path = '/content/drive/MyDrive/archive.zip'
extract_path = '/content/dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


Mounted at /content/drive


In [4]:
#سلول 3:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import random
import pandas as pd
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model = AutoModelForMaskedLM.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# بارگذاری داده اصلی
train_df = pd.read_csv('/content/dataset/train.csv')
train_df['words'] = train_df['words'].apply(lambda x: x.strip().split())
train_df['words_label'] = train_df['words_label'].apply(lambda x: x.strip().split())

def mask_and_predict(words, labels):
    new_words = []
    new_labels = labels.copy()

    for i, (word, label) in enumerate(zip(words, labels)):
        if label == "O" and word.isalpha() and random.random() < 0.3:
            temp_words = words.copy()
            temp_words[i] = tokenizer.mask_token
            text = " ".join(temp_words)
            inputs = tokenizer.encode(text, return_tensors="pt").to(device)
            mask_index = (inputs == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
            if len(mask_index) == 0:
                continue
            with torch.no_grad():
                outputs = model(inputs)
            logits = outputs.logits
            predicted_token_id = torch.argmax(logits[0, mask_index.item()]).item()
            predicted_token = tokenizer.decode([predicted_token_id]).strip()
            if predicted_token and predicted_token != tokenizer.mask_token:
                new_words.append(predicted_token)
            else:
                new_words.append(word)
        else:
            new_words.append(word)
    return new_words, new_labels

# تولید 1000 داده جدید
augmented_data = []

for _ in tqdm(range(1000), desc="Generating data"):
    row = train_df.sample(n=1).iloc[0]
    words = row['words']
    labels = row['words_label']
    intent = row['intent_label']

    new_words, new_labels = mask_and_predict(words, labels)
    augmented_data.append({
        "words": " ".join(new_words),
        "words_label": " ".join(new_labels),
        "intent_label": intent
    })

aug_df = pd.DataFrame(augmented_data)
aug_df.to_csv('/content/augmented_train.csv', index=False)

print("داده‌های افزوده‌شده با ParsBERT با موفقیت ذخیره شدند.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/654M [00:00<?, ?B/s]


Generating data:   0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)

Generating data: 100%|██████████| 1000/1000 [00:22<00:00, 43.55it/s]

داده‌های افزوده‌شده با ParsBERT با موفقیت ذخیره شدند.


In [5]:
#سلول4
aug_df = pd.read_csv('/content/augmented_train.csv')
aug_df.head(10)


,words,words_label,intent_label
0,برام یه عکس دو تا هتل داریوش,O O O O O B-Destination I-Destination,Catch_a_Cab
1,پیش بینی هوا شناسی اشتباه :,O O O O O O,Get_Weather
2,آب و هوا فردا بارانیه، اما چتر یادت نره,O O O B-Date O O O O O,Get_Weather
3,و برای شام سه تا ماکارونی سفارش .,O O O B-Number I-Number B-Food O O,Order_Food
4,این ترانه از جنیفر لوپز پخش شد,O O O B-Artist I-Artist O O,Play_Music
5,دمای حرارت در ملبورن چند درجه سانتیگراد است,O O O B-Location O O O O,Get_Weather
6,یه آلبوم موسیقی محبوب پخش کن,O O O O O O,Play_Music
7,همین الان هوا سرد است,B-Date I-Date O O O,Get_Weather
8,پنجشنبه آینده نیز دما در اهواز چند درجه خواهد است,B-Date I-Date O O O B-Location O O O O,Get_Weather
9,پیش بینی هوا شناسی چه جوریه,O O O O O O,Get_Weather


In [6]:
#سلول5:
original_df = pd.read_csv('/content/dataset/train.csv')
if 'Unnamed: 0' in original_df.columns:
    original_df = original_df.drop(columns=['Unnamed: 0'])

combined_df = pd.concat([original_df, aug_df], ignore_index=True)
combined_df.to_csv('/content/train_augmented_combined.csv', index=False)
combined_df.sample(10)


,words,words_label,intent_label
915,قیمت آژانس از خونه تا شاهچراغ چند برابر است,O O O B-Departure O B-Destination O O O,Catch_a_Cab
603,این ترانه از محسن چاووشی پخش شد,O O O B-Artist I-Artist O O,Play_Music
1099,پیش بینی اسیب شناسی دمشق شنبه قبل چطور بود,O O O O B-Location B-Date I-Date O O,Get_Weather
883,بهترین موسیقی مایکل جکسون رو پخش کن,O O B-Artist I-Artist O O O,Play_Music
431,آژانس از باشگاه تا تجریش چنده,O O B-Departure O B-Destination O,Catch_a_Cab
236,یک ترانه پرفروش از کیتی پری پخش کن,O O O O B-Artist I-Artist O O,Play_Music
381,پس فردا در قاهره پیش بینی آب و هوا چگونه است,B-Date I-Date O B-Location O O O O O O O,Get_Weather
646,اب و هوا فردا بارانیه، لطفا چتر یادت نره,O O O B-Date O O O O O,Get_Weather
332,من برای ناهار میخوام یک عدد همبرگر بخورم از ...,O O O O B-Number I-Number B-Food O O B-Restaur...,Order_Food
1267,برای ناهار از ساندویچ آیدا چهار تا کباب کوبیده...,O O O B-Restaurant I-Restaurant B-Number I-Num...,Order_Food


In [7]:
#سلول 6:
import torch

class Config:
    def __init__(self):
        self.model_name = "HooshvareLab/bert-base-parsbert-uncased"
        self.max_length = 32
        self.batch_size = 16
        self.learning_rate = 5e-5
        self.epochs = 10
        self.dropout = 0.3

config = Config()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(42)


In [8]:
#سلول 7:
import pandas as pd

def load_data():
    # خواندن داده‌ها
    train_df = pd.read_csv('/content/train_augmented_combined.csv')
    test_df = pd.read_csv('/content/dataset/test.csv')

    # حذف ردیف‌های ناقص
    train_df.dropna(subset=['words', 'words_label'], inplace=True)
    test_df.dropna(subset=['words', 'words_label'], inplace=True)

    def split_row(row):
        words = str(row['words']).split()
        labels = str(row['words_label']).split()
        return words, labels

    train_df['tokens'], train_df['ner_tags'] = zip(*train_df.apply(split_row, axis=1))
    test_df['tokens'], test_df['ner_tags'] = zip(*test_df.apply(split_row, axis=1))

    unique_intents = sorted(train_df['intent_label'].unique())
    unique_ner_tags = sorted(set(tag for tags in train_df['ner_tags'] for tag in tags))

    intent2id = {v: k for k, v in enumerate(unique_intents)}
    intent_id2label = {k: v for v, k in intent2id.items()}

    ner_tag2id = {v: k for k, v in enumerate(unique_ner_tags)}
    ner_id2tag = {k: v for v, k in ner_tag2id.items()}

    return train_df.reset_index(drop=True), test_df.reset_index(drop=True), intent2id, intent_id2label, ner_tag2id, ner_id2tag

train_df, test_df, intent2id, intent_id2label, ner_tag2id, ner_id2tag = load_data()


In [9]:
#سلول 8:
from transformers import AutoTokenizer
from torch.utils.data import Dataset

tokenizer = AutoTokenizer.from_pretrained(config.model_name)

class IntentDataset(Dataset):
    def __init__(self, df):
        self.texts = [" ".join(tokens) for tokens in df['tokens']]
        self.labels = [intent2id[i] for i in df['intent_label']]

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=config.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(self.labels[idx])
        }

class NERDataset(Dataset):
    def __init__(self, df):
        self.tokens = df['tokens']
        self.labels = df['ner_tags']

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        tokens = self.tokens[idx]
        ner_tags = self.labels[idx]

        encoding = tokenizer(
            tokens,
            is_split_into_words=True,
            padding='max_length',
            truncation=True,
            max_length=config.max_length,
            return_tensors='pt'
        )

        labels = [-100] * config.max_length
        word_ids = encoding.word_ids()

        for i, word_idx in enumerate(word_ids):
            if word_idx is not None and word_idx < len(ner_tags):
                labels[i] = ner_tag2id[ner_tags[word_idx]]

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(labels)
        }


In [10]:
#سلول9
from transformers import BertForSequenceClassification, get_scheduler
from torch.optim import AdamW
from torch.utils.data import DataLoader

intent_model = BertForSequenceClassification.from_pretrained(
    config.model_name,
    num_labels=len(intent2id)
)
intent_model.to(device)

train_loader = DataLoader(IntentDataset(train_df), batch_size=config.batch_size, shuffle=True)

optimizer = AdamW(intent_model.parameters(), lr=config.learning_rate, weight_decay=0.01)
lr_scheduler = get_scheduler("linear", optimizer, num_warmup_steps=0,
                             num_training_steps=config.epochs * len(train_loader))

intent_model.train()
for epoch in range(config.epochs):
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = intent_model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Intent Loss: {total_loss/len(train_loader):.4f}")

torch.save(intent_model.state_dict(), "intent_model.pth")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch 1, Intent Loss: 0.1161
Epoch 2, Intent Loss: 0.0031
Epoch 3, Intent Loss: 0.0015
Epoch 4, Intent Loss: 0.0010
Epoch 5, Intent Loss: 0.0008
Epoch 6, Intent Loss: 0.0006
Epoch 7, Intent Loss: 0.0006
Epoch 8, Intent Loss: 0.0005
Epoch 9, Intent Loss: 0.0005
Epoch 10, Intent Loss: 0.0005


In [11]:
#سلول 10:
from transformers import BertForTokenClassification
from torch.utils.data import DataLoader

ner_model = BertForTokenClassification.from_pretrained(
    config.model_name,
    num_labels=len(ner_tag2id)
)
ner_model.to(device)

ner_loader = DataLoader(NERDataset(train_df), batch_size=config.batch_size, shuffle=True)

optimizer = AdamW(ner_model.parameters(), lr=config.learning_rate, weight_decay=0.01)
lr_scheduler = get_scheduler("linear", optimizer, num_warmup_steps=0,
                             num_training_steps=config.epochs * len(ner_loader))

ner_model.train()
for epoch in range(config.epochs):
    total_loss = 0
    for batch in ner_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = ner_model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, NER Loss: {total_loss/len(ner_loader):.4f}")

torch.save(ner_model.state_dict(), "ner_model.pth")


Some weights of BertForTokenClassification were not initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, NER Loss: 0.3360
Epoch 2, NER Loss: 0.0223
Epoch 3, NER Loss: 0.0148
Epoch 4, NER Loss: 0.0062
Epoch 5, NER Loss: 0.0036
Epoch 6, NER Loss: 0.0019
Epoch 7, NER Loss: 0.0009
Epoch 8, NER Loss: 0.0009
Epoch 9, NER Loss: 0.0007
Epoch 10, NER Loss: 0.0006


In [12]:
#سلول11:
from sklearn.metrics import classification_report
from seqeval.metrics import classification_report as seqeval_report

def evaluate_intent(model, df):
    model.eval()
    loader = DataLoader(IntentDataset(df), batch_size=config.batch_size)
    preds, labels = [], []
    with torch.no_grad():
        for batch in loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=1)
            preds.extend(pred.cpu().numpy())
            labels.extend(batch['labels'].cpu().numpy())
    print("\n Intent Classification Report:")
    print(classification_report(labels, preds, target_names=intent_id2label.values()))

def evaluate_ner(model, df):
    model.eval()
    loader = DataLoader(NERDataset(df), batch_size=1)
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=2)

            true_labels, pred_labels = [], []
            for i in range(config.max_length):
                if labels[0][i] != -100:
                    true_labels.append(ner_id2tag[labels[0][i].item()])
                    pred_labels.append(ner_id2tag[preds[0][i].item()])

            all_labels.append(true_labels)
            all_preds.append(pred_labels)

    print("NER Report:")
    print(seqeval_report(all_labels, all_preds))

evaluate_intent(intent_model, test_df)
evaluate_ner(ner_model, test_df)


 Intent Classification Report:
              precision    recall  f1-score   support

 Catch_a_Cab       1.00      1.00      1.00        31
 Get_Weather       1.00      1.00      1.00        31
  Order_Food       1.00      1.00      1.00        29
  Play_Music       1.00      1.00      1.00        31

    accuracy                           1.00       122
   macro avg       1.00      1.00      1.00       122
weighted avg       1.00      1.00      1.00       122



/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


NER Report:
              precision    recall  f1-score   support

      Artist       1.00      1.00      1.00        25
        Date       1.00      0.95      0.98        21
   Departure       1.00      1.00      1.00        12
 Destination       1.00      1.00      1.00        31
        Food       1.00      1.00      1.00        29
    Location       1.00      1.00      1.00        22
      Number       1.00      1.00      1.00        29
  Restaurant       1.00      1.00      1.00        30

   micro avg       1.00      0.99      1.00       199
   macro avg       1.00      0.99      1.00       199
weighted avg       1.00      0.99      1.00       199



In [13]:
#سلول 12:
def predict_intent(text):
    encoding = tokenizer(text, padding='max_length', truncation=True,
                         max_length=config.max_length, return_tensors='pt')
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    intent_model.eval()
    with torch.no_grad():
        output = intent_model(input_ids=input_ids, attention_mask=attention_mask)
        pred = torch.argmax(output.logits, dim=1).item()
        return intent_id2label[pred]

def predict_ner(text):
    tokens = text.split()
    encoding = tokenizer(tokens, is_split_into_words=True,
                         padding='max_length', truncation=True,
                         max_length=config.max_length, return_tensors='pt')
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    word_ids = encoding.word_ids()

    ner_model.eval()
    with torch.no_grad():
        outputs = ner_model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=2)[0].cpu().numpy()

    final_tags = []
    current_word = None
    for i, word_idx in enumerate(word_ids):
        if word_idx is not None and word_idx != current_word:
            word = tokens[word_idx]
            tag = ner_id2tag[predictions[i]]
            final_tags.append(f"{word}: {tag}")
            current_word = word_idx

    return final_tags

# مثال تستی
sample_texts = [
    "تا مهرشهر میخوام تاکسی بگیرم",
    "فردا هوا در کرج چطوره",
    "از رستوران هرما دو تا پیتزا و یک پاستا میخوام",
    "آهنگ جدید بیلی آیلیش رو پخش کن"
]

for text in sample_texts:
    intent = predict_intent(text)
    tags = predict_ner(text)
    print(f"\nمتن: {text}")
    print(f"Intent: {intent}")
    print("NER Tags:")
    for tag in tags:
        print(" ", tag)



متن: تا مهرشهر میخوام تاکسی بگیرم
Intent: Catch_a_Cab
NER Tags:
  تا: O
  مهرشهر: B-Destination
  میخوام: O
  تاکسی: O
  بگیرم: O

متن: فردا هوا در کرج چطوره
Intent: Get_Weather
NER Tags:
  فردا: B-Date
  هوا: O
  در: O
  کرج: B-Location
  چطوره: O

متن: از رستوران هرما دو تا پیتزا و یک پاستا میخوام
Intent: Order_Food
NER Tags:
  از: O
  رستوران: B-Restaurant
  هرما: I-Restaurant
  دو: B-Number
  تا: I-Number
  پیتزا: B-Food
  و: B-Number
  یک: I-Number
  پاستا: B-Food
  میخوام: O

متن: آهنگ جدید بیلی آیلیش رو پخش کن
Intent: Play_Music
NER Tags:
  آهنگ: O
  جدید: O
  بیلی: B-Artist
  آیلیش: I-Artist
  رو: O
  پخش: O
  کن: O


In [14]:
import gradio as gr
import torch
from transformers import AutoTokenizer

def predict_intent(text):
    encoding = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=config.max_length,
        return_tensors='pt'
    ).to(device)

    with torch.no_grad():
        outputs = intent_model(**encoding)
        pred = torch.argmax(outputs.logits, dim=1).item()
        return intent_id2label[pred]

def predict_ner(text):
    intent = predict_intent(text)
    tokens = text.split()

    SPECIAL_RULES = {
        "و": "O", "های": "O", "رو": "O", "از": "O", "به": "O", "تا": "O", "را": "O", "ها": "O",
        "Catch_a_Cab": {"از": "O", "به": "O", "میخوام": "O", "بگیرم": "O"},
        "Order_Food": {"و": "O", "سفارش": "O", "بده": "O", "میخوام": "O"},
        "Play_Music": {"های": "O", "رو": "O", "از": "O", "پخش": "O", "کن": "O"},
        "Get_Weather": {"هوا": "O", "چطور": "O", "چگونه": "O", "است": "O"}
    }

    encoding = tokenizer(
        tokens,
        is_split_into_words=True,
        padding='max_length',
        truncation=True,
        max_length=config.max_length,
        return_tensors='pt'
    ).to(device)

    ner_model.eval()
    with torch.no_grad():
        outputs = ner_model(**encoding)

    predictions = torch.argmax(outputs.logits, dim=2)[0].cpu().numpy()
    word_ids = encoding.word_ids()

    results = []
    current_word = None
    for i, word_idx in enumerate(word_ids):
        if word_idx is not None and word_idx != current_word:
            word = tokens[word_idx]
            original_tag = ner_id2tag[predictions[i]]
            tag = SPECIAL_RULES.get(word, SPECIAL_RULES.get(intent, {}).get(word, original_tag))

            if intent == "Catch_a_Cab" and word in ["میدان", "بلوار", "خیابان"] and word_idx+1 < len(tokens):
                next_word = tokens[word_idx+1]
                results.append(f"{word} {next_word}: B-Destination")
                current_word = word_idx + 1
                continue

            results.append(f"{word}: {tag}")
            current_word = word_idx

    return intent, "\n".join(results)

with gr.Blocks(title="دستیار هوشمند فارسی") as demo:
    gr.Markdown("## دستیار هوشمند فارسی")

    with gr.Row():
        with gr.Column():
            input_text = gr.Textbox(label="درخواست خود را وارد کنید",
                                 placeholder="مثال: آهنگ های ابی را پخش کن")
            with gr.Row():
                analyze_btn = gr.Button("آنالیز کن", variant="primary")
                clear_btn = gr.Button("پاک کردن", variant="secondary")

        with gr.Column():
            intent_output = gr.Textbox(label="نوع درخواست")
            ner_output = gr.Textbox(label="موجودیت‌های شناسایی شده",
                                 interactive=False)

    examples = gr.Examples(
        examples=[
            ["آهنگ جدید بیلی آیلیش رو پخش کن"],
            ["هوای فردا در تهران چطوره؟"],
            ["از میدان انقلاب به تجریش تاکسی می‌خواهم"]
        ],
        inputs=input_text
    )

    analyze_btn.click(
        fn=predict_ner,
        inputs=input_text,
        outputs=[intent_output, ner_output]
    )

    clear_btn.click(
        fn=lambda: ["", "", ""],
        outputs=[input_text, intent_output, ner_output],
        queue=False
    )

if __name__ == "__main__":
    demo.launch(inline=True)



It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5d2a29195a900a413b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
